In [1]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=8a1932c8d050b36bc71c05bc17b5d90fd366b7ccfef3b848e3c5deb84a65ba89
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [2]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [3]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [4]:
# calling API 
data=YahooFinancials('XRP-USD').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
xrp_df = pd.DataFrame(data['XRP-USD']['prices'])

xrp_df =xrp_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
xrp_df.rename(columns={'formatted_date':'date'},inplace=True)
xrp_df['date']=pd.to_datetime(xrp_df['date'])

In [5]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [6]:
# Scale data for ML engine
scaler = MinMaxScaler()
xrp_df['scaled_close'] = scaler.fit_transform(np.expand_dims(xrp_df['close'].values, axis=1))

In [7]:
def PrepareData(days):
    df = xrp_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [8]:
PrepareData(3) # 3 days


(         close       date  scaled_close    future
 0     0.292431 2019-11-07      0.089901  0.082707
 1     0.276953 2019-11-08      0.080794  0.079696
 2     0.280196 2019-11-09      0.082702  0.078126
 3     0.280204 2019-11-10      0.082707  0.078490
 4     0.275086 2019-11-11      0.079696  0.076933
 ...        ...        ...           ...       ...
 1093  0.504404 2022-11-04      0.214620  0.191940
 1094  0.493330 2022-11-05      0.208105  0.156864
 1095  0.470670 2022-11-06      0.194772  0.113877
 1096  0.465857 2022-11-07      0.191940  0.150665
 1097  0.406242 2022-11-08      0.156864  0.142700
 
 [1098 rows x 4 columns], array([[0.18312475],
        [0.1856318 ],
        [0.21462038],
        [0.20810473],
        [0.1947722 ],
        [0.19194034],
        [0.15686446],
        [0.11387731],
        [0.15066478],
        [0.1426996 ]], dtype=float32), array([[[0.08990109916951765, Timestamp('2019-11-07 00:00:00')],
         [0.08079426472922697, Timestamp('2019-11-08 00:00:

In [9]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [10]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
137/137 [==============================] - 6s 13ms/step - loss: 0.0068
Epoch 2/80
137/137 [==============================] - 3s 21ms/step - loss: 0.0035
Epoch 3/80
137/137 [==============================] - 3s 22ms/step - loss: 0.0028
Epoch 4/80
137/137 [==============================] - 3s 20ms/step - loss: 0.0024
Epoch 5/80
137/137 [==============================] - 3s 22ms/step - loss: 0.0022
Epoch 6/80
137/137 [==============================] - 3s 24ms/step - loss: 0.0023
Epoch 7/80
137/137 [==============================] - 3s 21ms/step - loss: 0.0019
Epoch 8/80
137/137 [==============================] - 3s 23ms/step - loss: 0.0022
Epoch 9/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0019
Epoch 10/80
137/137 [==============================] - 2s 12ms/step - loss: 0.0021
Epoch 11/80
137/137 [==============================] - 2s 12ms/step - loss: 0.0016
Epoch 12/80
137/137 [==============================] - 2s 12ms/step - loss: 0.0016
Epoch 13/80
1

In [11]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (0.39$, 0.38$, 0.37$)


In [12]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
xrp_lstm=pd.DataFrame()
xrp_lstm['future days']=days
xrp_lstm['price']=predictions
xrp_lstm['ticker']='XRP'

xrp_lstm.to_csv('xrp_lstm.csv') 
files.download('xrp_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>